In [1]:
from funcoesTermosol import *
import pandas as pd
import numpy as np

In [2]:
# Funções para o cálculo da matriz de rigidez

def gauss_seidel(ite, tol, k, f):
    u = np.zeros((k.shape[0], 1))
    u_ant = np.zeros((k.shape[0], 1))
    for n in range(ite):
        for i in range(k.shape[0]): #percorre as linhas da matriz
            soma = 0
            for j in range(k.shape[1]): #percorre as colunas da matriz
                if i != j:
                    soma += k[i, j] * u[j]
            u[i] = (f[i] - soma) / k[i, i]
        erro = ((u - u_ant) / u).max(axis=0)
        if erro < tol:
            return u, n
        u_ant = np.copy(u)
    return u, n

def jacobi(ite, tol, k, f):
    u = np.zeros((k.shape[0], 1))
    u_ant = np.zeros((k.shape[0], 1))
    for n in range(ite):
        for i in range(k.shape[0]): #percorre as linhas da matriz
            soma = 0
            for j in range(k.shape[1]): #percorre as colunas da matriz
                if i != j:
                    soma += k[i, j] * u_ant[j]
            u[i] = (f[i] - soma) / k[i, i]
        erro = ((u - u_ant) / u).max(axis=0)
        if erro < tol:
            return u, n
        u_ant = np.copy(u)
    return u, n

In [3]:
[nn,N,nm,Inc,nc,F,nr,R] = importa('solucao.xls')
dados = [nn,N,nm,Inc,nc,F,nr,R] 
dados

[7,
 array([[0.   , 0.144, 0.192, 0.288, 0.384, 0.432, 0.576],
        [0.   , 0.072, 0.   , 0.144, 0.   , 0.072, 0.   ]]),
 11,
 array([[1.0000e+00, 2.0000e+00, 1.9314e+11, 5.2500e-06],
        [2.0000e+00, 4.0000e+00, 1.9314e+11, 5.2500e-06],
        [4.0000e+00, 6.0000e+00, 1.9314e+11, 5.2500e-06],
        [6.0000e+00, 7.0000e+00, 1.9314e+11, 5.2500e-06],
        [2.0000e+00, 3.0000e+00, 1.9314e+11, 5.2500e-06],
        [3.0000e+00, 4.0000e+00, 1.9314e+11, 5.2500e-06],
        [4.0000e+00, 5.0000e+00, 1.9314e+11, 5.2500e-06],
        [5.0000e+00, 6.0000e+00, 1.9314e+11, 5.2500e-06],
        [1.0000e+00, 3.0000e+00, 1.9314e+11, 5.2500e-06],
        [3.0000e+00, 5.0000e+00, 1.9314e+11, 5.2500e-06],
        [5.0000e+00, 7.0000e+00, 1.9314e+11, 5.2500e-06]]),
 6,
 array([[    0.],
        [    0.],
        [-1300.],
        [-1500.],
        [    0.],
        [    0.],
        [-1300.],
        [-1500.],
        [    0.],
        [    0.],
        [-1300.],
        [-1500.],
        [  

In [4]:
# Criação da matriz de rigidez global
Kg = np.zeros((2*nn,2*nn))

# Vetor de Comprimentos dos Membros
L_vet = np.zeros((nm,1))

# Vetor de Cossenos dos Membros
Cos_vet = np.zeros((nm,1))

# Vetor de Senos dos Membros
Sen_vet = np.zeros((nm,1))

for i in range(nm):
    n1 = int(Inc[i][0])
    n2 = int(Inc[i][1])
    x_n1 = N[0][n1-1]
    y_n1 = N[1][n1-1]

    x_n2 = N[0][n2-1]
    y_n2 = N[1][n2-1]

    L = np.sqrt((x_n2-x_n1)**2 + (y_n2-y_n1)**2)

    # Definição de vetores importantes
    L_vet[i] = L

    # Cálculo de cosseno e seno
    c = (x_n2-x_n1)/L
    s = (y_n2-y_n1)/L

    # Definição de vetores importantes
    Cos_vet[i] = c
    Sen_vet[i] = s

    E = Inc[i][2]
    A = Inc[i][3]

    k = (E*A)/L * np.array([[c**2, c*s, -c**2, -c*s],  # matriz de rigidez local
                            [c*s, s**2, -c*s, -s**2],
                            [-c**2, -c*s, c**2, c*s],
                            [-c*s, -s**2, c*s, s**2]])

    k = np.array(k)

    indices = ([2*n1-2, 2*n1-1, 2*n2-2, 2*n2-1])
    print(indices)

    for i in range(4):
        for j in range(4):
            Kg[indices[i], indices[j]] += k[i, j]
    
print(f'\nMatriz de rigidez global (Kg):\n{Kg}')
Kg.shape


[0, 1, 2, 3]
[2, 3, 6, 7]
[6, 7, 10, 11]
[10, 11, 12, 13]
[2, 3, 4, 5]
[4, 5, 6, 7]
[6, 7, 8, 9]
[8, 9, 10, 11]
[0, 1, 4, 5]
[4, 5, 8, 9]
[8, 9, 12, 13]

Matriz de rigidez global (Kg):
[[ 1.03197038e+07  2.51926599e+06 -5.03853197e+06 -2.51926599e+06
  -5.28117188e+06  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 2.51926599e+06  1.25963299e+06 -2.51926599e+06 -1.25963299e+06
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-5.03853197e+06 -2.51926599e+06  1.36825619e+07 -3.69714935e+05
  -3.60549794e+06  5.40824691e+06 -5.03853197e+06 -2.51926599e+06
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-2.51926599e+06 -1.25963299e+06 -3.69714935e+05  1.06316363e+07
   5.40824691e+06 -8.11237036e+06 -2.51926599e+06 

(14, 14)

In [5]:
# Remover linhas dos graus de liberdade restritos
R = [int(i[0]) for i in R.tolist()]
Kg_r = np.delete(Kg, R, axis=0)
F_r = np.delete(F, R, axis=0)

# Remover colunas dos graus de liberdade restritos
Kg_r = np.delete(Kg_r, R, axis=1)

Kg_r, F_r, R

(array([[ 1.36825619e+07, -3.69714935e+05, -3.60549794e+06,
          5.40824691e+06, -5.03853197e+06, -2.51926599e+06,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [-3.69714935e+05,  1.06316363e+07,  5.40824691e+06,
         -8.11237036e+06, -2.51926599e+06, -1.25963299e+06,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [-3.60549794e+06,  5.40824691e+06,  1.59705907e+07,
         -2.70412345e+06, -1.80274897e+06, -2.70412345e+06,
         -5.28117188e+06,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 5.40824691e+06, -8.11237036e+06, -2.70412345e+06,
          1.21685555e+07, -2.70412345e+06, -4.05618518e+06,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [-5.03853197e+06, -2.51926599e+06, -1.80274897e+06,
         -2.70412345e+06,  1.36825619e+07,  

In [6]:
# Resolver o sistema de equações para obter os deslocamentos nodais
ite = 100
tol = 1e-20
u_j, n_j = jacobi(ite, tol, Kg_r, F_r)
u_gs, n_gs = gauss_seidel(ite, tol, Kg_r, F_r)

print("Método de Jacobi")
print(u_j, n_j)
print("-----------------------------")
print("Método de Gauss-Seidel")
print(u_gs, n_gs)

Método de Jacobi
[[-107.00362006]
 [ 166.91398419]
 [ 128.33301721]
 [-179.87677309]
 [  12.374108  ]
 [  62.90165329]
 [-155.74563625]
 [-198.27817274]
 [  89.19229259]
 [ 210.28944153]
 [  74.56133209]] 99
-----------------------------
Método de Gauss-Seidel
[[ 0.00089023]
 [-0.00399389]
 [ 0.00029967]
 [-0.00431891]
 [ 0.00018111]
 [-0.00410975]
 [ 0.00045672]
 [-0.00431026]
 [-0.0003758 ]
 [-0.00395415]
 [ 0.00101554]] 99


C:\Users\arthu\AppData\Local\Temp\ipykernel_23972\3022799297.py:29: RuntimeWarning: invalid value encountered in divide
  erro = ((u - u_ant) / u).max(axis=0)


In [8]:
# Construção da matriz de deslocamentos nodais
u_desl = np.zeros((2*nn, 1))

c = 0
for i in range(2*nn):
    if i not in R:
        # Condição para extrair os dados da solução de Gauss-Seidel de forma geral
        u_desl[i] = u_gs[c]
        c += 1
    else:
        u_desl[i] = 0

print(f"Deslocamentos [m]:\n{u_desl}")

Deslocamentos [m]:
[[ 0.        ]
 [ 0.        ]
 [ 0.00100474]
 [-0.00431173]
 [ 0.00035977]
 [-0.00466313]
 [ 0.00025974]
 [-0.00443928]
 [ 0.00055859]
 [-0.00463759]
 [-0.00032427]
 [-0.00424778]
 [ 0.00116451]
 [ 0.        ]]


In [9]:
# Calculo das reações de apoio	

# Kg * u = Reações de apoio (multiplicação de matrizes) 6x6 * 6x1 = 6x1

# Construção da matriz de reações de apoio
R_apoio = np.zeros((2*nn, 1))

# Realiza a multiplicação de matrizes para obter as reações de apoio 
R_apoio = np.dot(Kg, u_desl)

print(f" Vetor de reações de apoio:\n{R_apoio}")

# As reações de apoio desejadas dependem do vetor de restricoes R
R_apoio_vet = np.zeros((len(R), 1))
c = 0
for r in R:
    R_apoio_vet[c] = R_apoio[r]
    c += 1
    
print(f"\n Reações de apoio [N]: \n{R_apoio_vet}")

 Vetor de reações de apoio:
[[ 3.90000000e+03]
 [ 2.90000000e+03]
 [-1.30000000e+03]
 [-1.50000000e+03]
 [-4.09272616e-12]
 [-1.36424205e-11]
 [-1.30000000e+03]
 [-1.50000000e+03]
 [ 2.72848411e-12]
 [-1.81898940e-12]
 [-1.30000000e+03]
 [-1.50000000e+03]
 [ 0.00000000e+00]
 [ 1.60000000e+03]]

 Reações de apoio [N]: 
[[3900.]
 [2900.]
 [1600.]]


In [10]:
# Calculo das Deformações, Tensões e Forças Internas nos Elementos

# Inicializa a matriz de deformações
Def_vet = np.zeros((nm,1))

# Inicializa a matriz de tensões
Ten_vet = np.zeros((nm,1))

# Inicializa a matriz de forças
F_vet = np.zeros((nm,1))

# Extração dos módulos de elasticidade e áreas dos elementos
for i in range(nm):
    # Nós do elemento
    n1 = int(Inc[i][0])
    n2 = int(Inc[i][1])

    # Módulo de elasticidade
    E = Inc[i][2]
    L = L_vet[i]

    indices = ([2*n1-2, 2*n1-1, 2*n2-2, 2*n2-1])
    #print(indices)

    # Matriz de deformação do elemento
    vetor_cos_sen = np.transpose(np.array([-Cos_vet[i], -Sen_vet[i], Cos_vet[i], Sen_vet[i]]))
    Def = (1/L) * np.dot(vetor_cos_sen, u_desl[indices])
    Def_vet[i] = Def

    # Matriz de tensão do elemento
    Ten_vet[i] = E * Def

    # Tensão = F / A -> F = E * A * Def
    A = Inc[i][3]
    F_vet[i] = E * A * Def

#print(np.transpose(np.array([-Cos_vet[i], -Sen_vet[i], Cos_vet[i], Sen_vet[i]])).shape)
#print(u_desl[indices].shape)
print(f"Deformações[] \n {Def_vet} \n")
print(f"Tensões internas [Pa] \n {Ten_vet} \n")
print(f"Forças internas [N] \n {F_vet} \n")

Deformações[] 
 [[-0.00639516]
 [-0.00449315]
 [-0.00377645]
 [-0.00352836]
 [-0.00075561]
 [ 0.00075561]
 [ 0.00191125]
 [-0.00191125]
 [ 0.00187379]
 [ 0.00103552]
 [ 0.00315587]] 

Tensões internas [Pa] 
 [[-1.23516136e+09]
 [-8.67807334e+08]
 [-7.29384078e+08]
 [-6.81468336e+08]
 [-1.45938980e+08]
 [ 1.45938980e+08]
 [ 3.69139773e+08]
 [-3.69139773e+08]
 [ 3.61904762e+08]
 [ 2.00000000e+08]
 [ 6.09523810e+08]] 

Forças internas [N] 
 [[-6484.59713475]
 [-4555.98850416]
 [-3829.26641147]
 [-3577.708764  ]
 [ -766.17964604]
 [  766.17964604]
 [ 1937.98381056]
 [-1937.98381056]
 [ 1900.        ]
 [ 1050.        ]
 [ 3200.        ]] 



In [11]:
# Uso da função exporta para gerar o arquivo de saída
geraSaida('solucao', R_apoio_vet, u_desl, Def_vet, F_vet, Ten_vet)